In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/03 10:15:02 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.129 instead (on interface wlp4s0)
23/03/03 10:15:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 10:15:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/03 10:15:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.3.2'

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz
!gunzip fhvhv_tripdata_2021-06.csv.gz

In [31]:
!ls -lh fhvhv_tripdata_2021-06.csv

-rw-rw-r-- 1 michal michal 878M Dec 20 01:13 fhvhv_tripdata_2021-06.csv


In [38]:
!head fhvhv_tripdata_2021-06.csv

dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,
B02510,2021-06-01 00:18:15,2021-06-01 00:25:47,49,17,N,
B02510,2021-06-01 00:33:06,2021-06-01 00:42:46,49,225,N,
B02510,2021-06-01 00:46:27,2021-06-01 00:56:50,225,177,N,
B02764,2021-06-01 00:48:06,2021-06-01 01:04:10,209,45,N,B02764


In [34]:
schema = types.StructType([
    #types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [35]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv')

df = df.repartition(12)

df.write.parquet('data/pq/fhvhv/2021/06/')

23/03/03 10:53:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/03 10:53:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/03 10:53:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/03 10:53:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/03 10:53:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/03/03 10:53:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/03 10:53:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/03 10:53:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/03 10:53:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [36]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [37]:
!ls -al --block-size=M data/pq/fhvhv/2021/06/*.parquet

-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021/06/part-00000-90aef90d-98c3-4ac6-93f9-4073e7e2afc4-c000.snappy.parquet
-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021/06/part-00001-90aef90d-98c3-4ac6-93f9-4073e7e2afc4-c000.snappy.parquet
-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021/06/part-00002-90aef90d-98c3-4ac6-93f9-4073e7e2afc4-c000.snappy.parquet
-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021/06/part-00003-90aef90d-98c3-4ac6-93f9-4073e7e2afc4-c000.snappy.parquet
-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021/06/part-00004-90aef90d-98c3-4ac6-93f9-4073e7e2afc4-c000.snappy.parquet
-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021/06/part-00005-90aef90d-98c3-4ac6-93f9-4073e7e2afc4-c000.snappy.parquet
-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021/06/part-00006-90aef90d-98c3-4ac6-93f9-4073e7e2afc4-c000.snappy.parquet
-rw-r--r-- 1 michal michal 22M Mar  3 10:53 data/pq/fhvhv/2021

In [39]:
df = spark.read.parquet('data/pq/fhvhv/2021/06/')

**Q3**: How many taxi trips were there on June 15?

In [40]:
from pyspark.sql import functions as F

In [41]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [42]:
df.registerTempTable('fhvhv_2021_06')

/home/michal/.virtualenvs/data-engineering-zoomcamp/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [43]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_06
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



**Q4**: Longest trip for each day

In [44]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [45]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02866|2021-06-01 09:03:41|2021-06-01 09:20:24|         211|         246|      N|                B02866|
|              B02887|2021-06-02 17:48:58|2021-06-02 18:05:18|          33|         144|      N|                B02887|
|              B02510|2021-06-02 22:59:33|2021-06-02 23:10:16|         229|         230|      N|                  null|
|              B02869|2021-06-01 08:43:08|2021-06-01 08:46:31|          74|         168|      N|                B02869|
|              B02510|2021-06-01 05:51:37|2021-06-01 06:11:33|          97|         161|      N|                  null|
+--------------------+------------------

In [49]:
df \
    .withColumn('duration_hours', (df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) / 3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration_hours')  \
    .orderBy('max(duration_hours)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------------+
|pickup_date|max(duration_hours)|
+-----------+-------------------+
| 2021-06-25|   66.8788888888889|
| 2021-06-22| 25.549722222222222|
| 2021-06-27| 19.980833333333333|
| 2021-06-26| 18.197222222222223|
| 2021-06-23| 16.466944444444444|
+-----------+-------------------+



In [52]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration_hours
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|    duration_hours|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
| 2021-06-24|13.909722222222221|
| 2021-06-04|             11.67|
| 2021-06-20|10.984444444444444|
| 2021-06-01|           10.2675|
| 2021-06-28| 9.966388888888888|
+-----------+------------------+



**Q5**: Most frequent `dispatching_base_num` ( old question )

How many stages this spark job has?



In [53]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 4126325|
|              B02764| 1355081|
|              B02872| 1153791|
|              B02875|  934990|
|              B02765|  697992|
+--------------------+--------+



In [ ]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

**Q6**: Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?</br>

In [54]:
df_zones = spark.read.parquet('zones')

In [55]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [56]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [57]:
df_zones.registerTempTable('zones')

/home/michal/.virtualenvs/data-engineering-zoomcamp/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [61]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1) AS PICKUP_COUNT
FROM
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+-------------------+------------+
|               Zone|PICKUP_COUNT|
+-------------------+------------+
|Crown Heights North|      231279|
|       East Village|      221244|
|        JFK Airport|      188867|
|     Bushwick South|      187929|
|      East New York|      186780|
+-------------------+------------+



In [ ]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_2021_02 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()